In [26]:
# Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np

In [2]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Twitter API Keys
consumer_key = "HQ3J09n34aybMU4DMybVVh4Lh"
consumer_secret = "dUBdEgL0iu84cNKW9Xu8FPlna3J3jx1nROpiEMbvSCUE6UrhS4"
access_token = "967430365215383553-y5IPz0wz252K0XNSsStJEiMCTqGEC4q"
access_token_secret = "6N9XI9WxrSVc7mUAXgspUQVJhqmfX14AsHoEJxgzJCKKT"

In [4]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [34]:
# Target User Accounts
target_users = ("@BBC", "@CBS", "@CNN", "@FoxNews", "@nytimes")

tweet_info = []
tweet_times = []
converted_timestamps = []

# Loop through each user
for user in target_users:
    tweetcount=100
    
    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 5 pages of tweets (total 100 tweets)
    for x in range(5):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(user, page=x)
        
        # Loop through all tweets
        for tweet in public_tweets:
            
            # Create variables
            user_name = tweet["user"]["screen_name"]
            tweet_text = tweet['text']
            raw_time = tweet["created_at"]
            
            tweet_times.append(raw_time)
            
            # Convert tweet timestamps to datetime objects that can be manipulated by Python
            for raw_time in tweet_times:
                converted_time = datetime.strptime(raw_time, "%a %b %d %H:%M:%S %z %Y")
                converted_timestamps.append(converted_time) 
        
            # Run Vader Analysis on each tweet
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]

            # Add each value to the appropriate array
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            
             # Append the tweet information into tweet_info list
            tweet_info.append({"Account": user_name,
                    "Text": tweet_text,
                    "Date": converted_time,
                    "Compound Score": compound,
                    "Positive Score": pos,
                    "Neutral Score": neu,
                    "Negative Score": neg,
                    "Tweets Ago": tweetcount})
            tweetcount-=1

In [35]:
# Create a Dataframe
tweet_info_pd = pd.DataFrame(tweet_info)
tweet_info_pd = tweet_info_pd[['Account', 'Text', 'Date', 'Compound Score', 'Positive Score', 'Neutral Score', 'Negative Score','Tweets Ago']]
tweet_info_pd.head()
#tweet_info_pd.to_csv('NewsMood.csv')

,Account,Text,Date,Compound Score,Positive Score,Neutral Score,Negative Score,Tweets Ago
0,BBC,'Harry was looking at his family for the first...,2018-03-07 19:12:02+00:00,0.0000,0.000,1.000,0.0,100
1,BBC,✂️ ‘My hair is a symbol of pride.’\n\n💇🏽💇🏿💇🏾 S...,2018-03-07 18:23:04+00:00,0.0000,0.000,1.000,0.0,99
2,BBC,The little mouse that inspired Pikachu is actu...,2018-03-07 17:42:01+00:00,0.4987,0.264,0.736,0.0,98
3,BBC,RT @BBCTwo: If you haven't seen #ACSVersace ye...,2018-03-07 17:27:29+00:00,0.0000,0.000,1.000,0.0,97
4,BBC,RT @BBCBreakfast: In many ways Hedy Lamarr was...,2018-03-07 17:22:51+00:00,0.0000,0.000,1.000,0.0,96


In [47]:
Media_Source=tweet_info_pd["Account"].unique()

for media in Media_Source:
    temp = tweet_info_pd[tweet_info_pd["Account"]==media]
    plt.scatter(temp["Tweets Ago"],temp['Compound Score'], marker='o', linewidth=0, alpha=0.8, label=media,
            facecolors=temp.Account.map({"@BBC": "r", "@CBS": "b", "@CNN": "g", "@FoxNews": "y", "@nytimes": "n"}))

tweet_info_pd.plot.scatter(user_name, compound)

# Incorporate the other graph properties
plt.title("Sentiment Analysis of Media Tweets (%s)" % time.strftime("%x"))
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.grid(True)

plt.xlim(101,0)
plt.ylim(-1,1)

# Save the figure
#plt.savefig("NMFig1.png")

# Show plot
plt.show()

ValueError: c of shape (100,) not acceptable as a color sequence for x with size 100, y with size 100

In [44]:
fox = plt.scatter(tweet_info_pd['@FoxNews']["Tweets Ago"], tweet_info_pd['@FoxNews']["Compound"] , marker="o", color="r", edgecolors="black", label = "Fox News")
bbc = plt.scatter(tweet_info_pd['@BBC']["Tweets Ago"], tweet_info_pd['@BBC']["Compound"] , marker="o", color="b", edgecolors="black", label = "BBC World")
cbs = plt.scatter(tweet_info_pd['@CBS']["Tweets Ago"], tweet_info_pd['@CBS']["Compound"] , marker="o", color="g", edgecolors="black", label = "CBS News")
nytimes = plt.scatter(tweet_info_pd['@nytimes']["Tweets Ago"], tweet_info_pd['@nytimes']["Compound"] , marker="o", color="y", edgecolors="black", label = "NY Times")
cnn = plt.scatter(tweet_info_pd['@CNN']["Tweets Ago"], tweet_info_pd['@CNN']["Compound"] , marker="o", color="m", edgecolors="black", label = "CNN")

plt.xlim(max(tweet_info_pd["@FoxNews"]["Tweets Ago"]) + 5, -5)
plt.ylim(-1,1)
plt.grid()

plt.title("Sentiment Analysis of Media Tweets 3/4/18")
plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")

plt.legend(handles=[fox, bbc, cbs, nytimes, cnn], loc='upper left', bbox_to_anchor=(1,1))

#plt.savefig("Sentiment Scatter")

plt.show()

KeyError: '@FoxNews'